# Single-View Geometry (Python)

## Usage
This code snippet provides an overall code structure and some interactive plot interfaces for the *Single-View Geometry* section of Assignment 3. In [main function](#Main-function), we outline the required functionalities step by step. Some of the functions which involves interactive plots are already provided, but [the rest](#Your-implementation) are left for you to implement.

## Package installation
- You will need [GUI backend](https://matplotlib.org/faq/usage_faq.html#what-is-a-backend) to enable interactive plots in `matplotlib`.
- In this code, we use `tkinter` package. Installation instruction can be found [here](https://anaconda.org/anaconda/tk).

# Common imports

In [1]:
# % matplotlib tk
import matplotlib.pyplot as plt
import numpy as np

from PIL import Image

# Provided functions

In [2]:
def get_input_lines(im, min_lines=3):
    """
    Allows user to input line segments; computes centers and directions.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        min_lines: minimum number of lines required
    Returns:
        n: number of lines from input
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        centers: np.ndarray of shape (3, n)
            where each column denotes the homogeneous coordinates of the centers
    """
    n = 0
    lines = np.zeros((3, 0))
    centers = np.zeros((3, 0))

    plt.figure()
    plt.imshow(im)
    print('Set at least %d lines to compute vanishing point' % min_lines)
    while True:
        print('Click the two endpoints, use the right key to undo, and use the middle key to stop input')
        clicked = plt.ginput(2, timeout=0, show_clicks=True)
        if not clicked or len(clicked) < 2:
            if n < min_lines:
                print('Need at least %d lines, you have %d now' % (min_lines, n))
                continue
            else:
                # Stop getting lines if number of lines is enough
                break

        # Unpack user inputs and save as homogeneous coordinates
        pt1 = np.array([clicked[0][0], clicked[0][1], 1])
        pt2 = np.array([clicked[1][0], clicked[1][1], 1])
        # Get line equation using cross product
        # Line equation: line[0] * x + line[1] * y + line[2] = 0
        line = np.cross(pt1, pt2)
        lines = np.append(lines, line.reshape((3, 1)), axis=1)
        # Get center coordinate of the line segment
        center = (pt1 + pt2) / 2
        centers = np.append(centers, center.reshape((3, 1)), axis=1)

        # Plot line segment
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='b')

        n += 1

    return n, lines, centers

In [3]:
def plot_lines_and_vp(im, lines, vp):
    """
    Plots user-input lines and the calculated vanishing point.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        vp: np.ndarray of shape (3, )
    """
    bx1 = min(1, vp[0] / vp[2]) - 10
    bx2 = max(im.shape[1], vp[0] / vp[2]) + 10
    by1 = min(1, vp[1] / vp[2]) - 10
    by2 = max(im.shape[0], vp[1] / vp[2]) + 10

    plt.figure()
    plt.imshow(im)
    for i in range(lines.shape[1]):
        if lines[0, i] < lines[1, i]:
            pt1 = np.cross(np.array([1, 0, -bx1]), lines[:, i])
            pt2 = np.cross(np.array([1, 0, -bx2]), lines[:, i])
        else:
            pt1 = np.cross(np.array([0, 1, -by1]), lines[:, i])
            pt2 = np.cross(np.array([0, 1, -by2]), lines[:, i])
        pt1 = pt1 / pt1[2]
        pt2 = pt2 / pt2[2]
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], 'g')

    plt.plot(vp[0] / vp[2], vp[1] / vp[2], 'ro')
    plt.show()

In [4]:
def get_top_and_bottom_coordinates(im, obj):
    """
    For a specific object, prompts user to record the top coordinate and the bottom coordinate in the image.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        obj: string, object name
    Returns:
        coord: np.ndarray of shape (3, 2)
            where coord[:, 0] is the homogeneous coordinate of the top of the object and coord[:, 1] is the homogeneous
            coordinate of the bottom
    """
    plt.figure()
    plt.imshow(im)

    print('Click on the top coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x1, y1 = clicked[0]
    # Uncomment this line to enable a vertical line to help align the two coordinates
    # plt.plot([x1, x1], [0, im.shape[0]], 'b')
    print('Click on the bottom coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x2, y2 = clicked[0]

    plt.plot([x1, x2], [y1, y2], 'b')

    return np.array([[x1, x2], [y1, y2], [1, 1]])

# Your implementation

In [5]:

"""
Solves for the vanishing point using the user-input lines.
"""
def get_vanishing_point(im,lines):
    h,w = im.shape[0],im.shape[1]
    vp = [0] * 3
    left_tmp = np.transpose( lines[:2,:] )
    right_tmp = np.transpose(-lines[2, :])
    vp[0],vp[1] = np.linalg.lstsq(left_tmp,right_tmp)[0]
    vp[2] = 1
    return vp
    # bx1 = min(1, vp[0]/vp[2])-10
    # bx2 = max(w, vp[0]/vp[2])+10
    # 
    # by1 = min(1, vp[1]/vp[2])-10
    # by2 = max(h, vp[1]/vp[2])+10
    # 
    # for i in range(lines.shape[1]):
    #     if lines[0,i] < lines[1,i]:
    #         point1 = (np.cross(np.transpose(np.array([1,0,-bx1])), lines[:, i]))
    #         point2 = (np.cross(np.transpose(np.array([1,0,-bx2])), lines[:, i]))
    #     else:
    #         point1 = (np.cross(np.transpose(np.array([1,0,-by1])), lines[:, i]))
    #         point2 = (np.cross(np.transpose(np.array([1,0,-by2])), lines[:, i]))
    #     point1 = point1/point1[2]
    #     point2 = point2/point2[2]
    #     
    #     
    # coordiante = [vp[0]/vp[2],vp[1]/vp[2]]
    

In [38]:
def get_horizon_line(vpts):
    """
    Calculates the ground horizon line.
    """
    horizon = np.real(np.cross(
        np.transpose(vpts[0,:]),
        np.transpose(vpts[1,:])
    ))
    print("origin H:",horizon)
    scale_factor = np.sqrt(horizon[0]**2+horizon[1]**2)
    return np.true_divide(horizon,scale_factor)

In [74]:
import matplotlib.lines as mlines
def plot_horizon_line(im, hline):
    print("horizontal line:",hline)
    print(im.shape)
    plt.figure()
    plt.imshow(im)
    # x = np.linspace(-5,5,100)
    # y = -hline[0]/hline[1]* x - hline[2]/hline[1]
    x1 = 0
    y1 = - hline[2]/hline[1]
    x2 = im.shape[1]
    y2 =  -hline[0]/hline[1]* x2 - hline[2]/hline[1]
    print([x1,y1 ], [x2, y2])
    
    plt.plot([x1, x2],[y1,y2],  'r')
    plt.show()
    """
    Plots the horizon line.
    """

In [145]:
import sympy
def get_camera_parameters(vpts):
    # vp_z =  np.transpose(np.array([-205, 213, 1]))
    # vp_x =  np.transpose(np.array([1295 ,222 ,1]))
    # vp_y =  np.transpose(np.array([428, 4683 ,1]))
    vp_x = vpts[0,:] 
    vp_y =  vpts[1,:] 
    vp_z =  vpts[2,:] 
    print("vp_X:",vp_x)
    print("vp_Y:",vp_y)
    print("vp_z:",vp_z)
    u = sympy.Symbol('u')
    v = sympy.Symbol('v')
    f = sympy.Symbol('f')
    sol = ( sympy.solve([-u*(vp_x[0]+vp_y[0]-vp_y[0]-vp_z[0]) +\
                 vp_x[0]*vp_y[0]-vp_y[0]*vp_z[0] + \
                 -v*(vp_x[1]+vp_y[1]-vp_y[1]-vp_z[1]) +\
                 vp_x[1]*vp_y[1]-vp_y[1]*vp_z[1] 
                ,
                  -u*(vp_z[0]+vp_x[0]-vp_y[0]-vp_z[0]) +\
                 vp_z[0]*vp_x[0]- vp_y[0]*vp_z[0] + \
                 -v*(vp_z[1]+vp_x[1]-vp_y[1]+vp_z[1]) +\
                 vp_z[1]*vp_x[1]-vp_y[1]*vp_z[1] 
                 ],[u,v]))
    print(sol)
    u_sol,v_sol = sol[u],sol[v]
    print("U_sol:",u_sol,type(u_sol))
    print("V_sol:",v_sol)
    u = sympy.Float(u_sol)
    v = sympy.Float(v_sol)
    f_sol = sympy.solve((u_sol-vp_x[0])*(u_sol-vp_y[0]) + (v_sol-vp_x[1]) * (v_sol - vp_y[1]) + f*f ,f )
    print("f_sol:",f_sol)
    f = sympy.Float(f_sol)
    f = f[0]
    return u,v,f
    

In [146]:
def get_rotation_matrix():
    """
    Computes the rotation matrix using the camera parameters.
    """
    # <YOUR IMPLEMENTATION>
    return R

In [147]:
def estimate_height():
    """
    Estimates height for a specific object using the recorded coordinates. You might need to plot additional images here for
    your report.
    """
    # <YOUR IMPLEMENTATION>
    pass

# Main function

In [149]:
CSL = "data\\part3\\CSL.jpg"
im = np.asarray(Image.open(CSL))

# Part 1
# Get vanishing points for each of the directions

num_vpts = 3
vpts = np.zeros((3, num_vpts))
# for i in range(num_vpts):
#     print('Getting vanishing point %d' % i)
#     # Get at least three lines from user input
#     n, lines, centers = get_input_lines(im)
#     # <YOUR IMPLEMENTATION> Solve for vanishing point
#     vpts[:, i] = get_vanishing_point(im,lines)
#     # Plot the lines and the vanishing point
#     plot_lines_and_vp(im, lines, vpts[:, i])

print(vpts)
vpts = np.array([
    [-202, 215 ,1],
    [1371 ,230 ,1],
    [503 ,4867, 1]
])
# plot horizon line
horizon_line = get_horizon_line(vpts)
plot_horizon_line(im, horizon_line)

# Part 2
# <YOUR IMPLEMENTATION> Solve for the camera parameters (f, u, v)

f, u, v = get_camera_parameters(vpts)
print("f:",f)
print("u:",u)
print("v:",v)
# 
# # Part 3
# # <YOUR IMPLEMENTATION> Solve for the rotation matrix
# R = get_rotation_matrix()
# 
# # Part 4
# # Record image coordinates for each object and store in map
# objects = ('person', 'CSL building', 'the spike statue', 'the lamp posts')
# coords = dict()
# for obj in objects:
#     coords[obj] = get_top_and_bottom_coordinates(im, obj)
# 
# # <YOUR IMPLEMENTATION> Estimate heights
# for obj in objects[1:]:
#     print('Estimating height of %s' % obj)
#     height = estimate_height()


[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
origin H: [    -15    1573 -341225]
horizontal line: [-9.53548509e-03  9.99954536e-01 -2.16916393e+02]
(685, 1024, 3)
[0, 216.9262555626192] [1024, 226.69103623649076]
vp_X: [-202  215    1]
vp_Y: [1371  230    1]
vp_z: [ 503 4867    1]
{u: 23813259333/14169491, v: 2594160175/14169491}
U_sol: 23813259333/14169491 <class 'sympy.core.numbers.Rational'>
V_sol: 2594160175/14169491
f_sol: [-3*sqrt(13035897751477386670)*I/14169491, 3*sqrt(13035897751477386670)*I/14169491]


TypeError: cannot create mpf from [-3*sqrt(13035897751477386670)*I/14169491, 3*sqrt(13035897751477386670)*I/14169491]